<a href="https://colab.research.google.com/github/bashusha/random_ml/blob/main/%D0%9E_%D0%BB%D0%B8%D0%BD%D0%B5%D0%B9%D0%BD%D0%BE%D0%B9_%D1%80%D0%B5%D0%B3%D1%80%D0%B5%D1%81%D1%81%D0%B8%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Это ноутбук ко встрече random_ml в дискорд-сообществе DataBreakfast https://discord.gg/sZkePhaWSZ . Сам по себе никаких секретных секретов не содержит, только иллюстрирует разговор о линейной регрессии.

In [ ]:
import numpy as np
import pandas as pd
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

# Загадываю вид зависимости

Мы будем в неожиданно выгодной позиции: в наших данных будет присутствовать настоящая зависимость, и иногда мы сможем туда подсматривать. В жизни, конечно же, никто такого не предоставляет и вообще не гарантирует: данные могут оказаться связаны нелинейно, совсем не связаны, местами отсутствовать или быть искажены. Но даже этот набор данных окажется не совсем прост.

In [ ]:
mapper = dict(enumerate('iabcd'))

In [ ]:
n_samples = 1000
rng = np.random.default_rng(seed=42)

df = pd.DataFrame(data=rng.random((n_samples)), columns=['x1'])
df['x2'] = np.sin(df['x1'])                                                     # переменные связаны, но нелинейно
df['x3'] = np.random.normal(size=n_samples)                                     # здесь будет шум, эту переменную не будем использовать в модели. сейчас он небольшой, а если станет большим?
df['x4'] = np.random.normal(size=n_samples, scale=.1)                           # здесь будет значимая переменная
df['x5'] = df['x1']*2 - 3                                                       # линейно зависимы!
df['c1'] = np.random.randint(size=n_samples, low=1, high=5)
df['c2'] = df['c1'].map(lambda x: mapper[x])

df['y'] = 5*df['x1']**2 + 2*df['x2'] + .3*df['x3']+df['x5']                     # эту зависимость я буду менять для разных экспериментов.
df.head()

,x1,x2,x3,x4,x5,c1,c2,y
0,0.773956,0.698970,-1.658718,-0.034541,-1.452088,2,b,2.443276
1,0.438878,0.424924,-0.248882,-0.082703,-2.122243,3,c,-0.383987
2,0.858598,0.756927,-0.239028,0.053734,-1.282804,1,a,3.845293
3,0.697368,0.642202,-1.326112,0.011304,-1.605264,2,b,1.712918
4,0.094177,0.094038,-0.281488,0.010775,-2.811645,3,c,-2.663668
...,...,...,...,...,...,...,...,...
995,0.437990,0.424120,-0.388695,-0.062078,-2.124021,1,a,-0.433215
996,0.639014,0.596404,-0.607781,0.044151,-1.721973,4,d,1.330193
997,0.002309,0.002309,0.323475,0.006607,-2.995382,4,d,-2.893696
998,0.993368,0.837869,-2.197951,0.010921,-1.013263,4,d,4.936996


От некоторых переменных таргет **в действительности** не зависит - но исследователь об этом не знает.  

Посмотрим на имеющиеся данные: пропусков нет, в числовых переменных записаны числа, и даже дисперсии отличаются всего на порядок. Наука говорит, что данные для линейных (и обобщенных линейных) моделей нужно масштабировать, но если этим этапом пренебречь - модель пока еще выдержит, хотя и будут некоторые неудобства (какие?)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x1      1000 non-null   float64
 1   x2      1000 non-null   float64
 2   x3      1000 non-null   float64
 3   x4      1000 non-null   float64
 4   x5      1000 non-null   float64
 5   c1      1000 non-null   int64  
 6   c2      1000 non-null   object 
 7   y       1000 non-null   float64
dtypes: float64(6), int64(1), object(1)
memory usage: 62.6+ KB


In [ ]:
df.describe()

,x1,x2,x3,x4,x5,c1,y
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000
mean,0.497178,0.456936,-0.009545,0.002821,-2.005644,2.51400,0.565979
std,0.291586,0.250535,0.942171,0.103668,0.583171,1.14066,2.575321
min,0.001233,0.001233,-2.807557,-0.408069,-2.997534,1.00000,-3.444139
25%,0.240259,0.237954,-0.652995,-0.064464,-2.519482,1.00000,-1.730245
50%,0.494824,0.474877,-0.060486,0.003066,-2.010352,3.00000,0.192649
75%,0.758893,0.688119,0.626991,0.073418,-1.482213,4.00000,2.683872
max,0.999105,0.840987,3.316519,0.326919,-1.001791,4.00000,6.163449


# Теперь будем отгадывать вид зависимости

Первым подходом будет подход "да что тут думать, тут прыгать надо", или "фит - предикт - и в отчет"

Мы предположим, что имеется линейная зависимость y (таргета) от иксов и c (независимых, или объясняющих переменных). И ведь мы (отчасти) правы - в нашей действительно имеющейся зависимости есть линейная часть. Но это неважно: это место произвола исследователя.

Первый шаг - стандартный. Разбиваем данные на train и test. Используем все доступные данные, кроме недоступной для наблюдения x3 и категориальных c2 - о них потом.

В экспериментах можно играть набором фичей: что-то удалять, что-то добавлять.

In [ ]:
X_train, X_test, y_train, y_test  = train_test_split(df.drop(columns=['y','x3','c2', ]), df['y'], test_size=0.2, random_state=1)

Возьмем готовую линейную регрессию, и в этом разговоре не будем уточнять, как она работает. Как ей пользоваться (что куда засунуть, а также что она про себя знает), описано в [документации](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) - там и примеры есть. Что еще надо? Бери да пользуйся ...

In [ ]:
regr = linear_model.LinearRegression()

In [ ]:
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)

In [ ]:
regr.coef_, regr.feature_names_in_

(array([ 4.96737182e+00, -1.88330677e+01, -2.35796780e-02,  9.93474365e+00,
         1.25066478e-02]),
 array(['x1', 'x2', 'x4', 'x5', 'c1'], dtype=object))

Собственно, регрессия подобрала какие-то коэффициенты и выдала предсказания, и даже предсказания не так плохи: R2 коэффициент 0.99 это не так мало. Все ли хорошо у этой регрессии? Куда посмотреть?

Имея возможность подглядывать в вид **настоящей зависимости**, можем ли дополнить мнение о подогнанной регрессии?

In [ ]:
# df['y'] = 5*df['x1']**2 + 2*df['x2'] + .3*df['x3']+df['x5']

print("Mean squared error: %.5f" % mean_squared_error(y_test, y_pred))

print("Coefficient of determination: %.5f" % r2_score(y_test, y_pred))

Mean squared error: 0.06768
Coefficient of determination: 0.98944


In [ ]:
regr.rank_

4

Что это за ранк такой? На встрече не комментировала, но может быть, он нам чем-то будет полезен?